In [1]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 7.0 MB/s eta 0:00:00


In [2]:
!pip install gpt4all

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.7 MB/s eta 0:00:00


In [5]:
!pip install gensim spacy transformers sentence-transformers scikit-learn faiss-cpu

In [6]:
from sentence_transformers import SentenceTransformer
from PyPDF2 import PdfReader
import torch

# Load a pre-trained or fine-tuned embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')  # Replace with your fine-tuned model if needed

# Preprocessing function to clean text
def preprocess_document(text):
    # Remove extra spaces, newlines, and handle encoding issues
    text = text.replace('\n', ' ').strip()
    return ' '.join(text.split())  # Normalize whitespace

# Function to read individual PDFs and add them to the corpus
def load_pdf_documents(pdf_paths):
    corpus = []
    for pdf_path in pdf_paths:
        reader = PdfReader(pdf_path)
        content = ""
        for page in reader.pages:
            if page.extract_text():
                content += page.extract_text()
        corpus.append(preprocess_document(content))
    return corpus

# List your PDF file paths
pdf_paths = [
    "/content/drive/MyDrive/Colab Notebooks/PaitentVitalMonitor.pdf",
    "/content/drive/MyDrive/Colab Notebooks/Machine Learning Enabled Vital Sign Monitoring System.pdf",
    "/content/drive/MyDrive/Colab Notebooks/Real_Time_Monitoring_of_Human_Body_Vital_Signs_usi.pdf",
    "/content/drive/MyDrive/Colab Notebooks/pulse_bp_respi_.pdf"
]

# Load and preprocess the PDFs
corpus = load_pdf_documents(pdf_paths)
print(f"Loaded {len(corpus)} documents from PDFs.")

# Generate embeddings
if corpus:  # Check if corpus is not empty
    document_embeddings = model.encode(corpus, convert_to_tensor=True)
    print("Embeddings shape:", document_embeddings.shape)
else:
    print("No valid content was loaded from the PDFs.")


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/PyPDF2/_cmap.py:142: PdfReadWarning: Advanced encoding /UniGB-UCS2-H not implemented yet
  warnings.warn(


Loaded 4 documents from PDFs.
Embeddings shape: torch.Size([4, 384])


In [7]:
import faiss
import numpy as np

# Convert embeddings to numpy array
document_embeddings_np = document_embeddings.cpu().detach().numpy()

# Create a FAISS index
dimension = document_embeddings_np.shape[1]  # Dimensionality of the embeddings
index = faiss.IndexFlatL2(dimension)  # L2 distance (Euclidean)
index.add(document_embeddings_np)  # Add document embeddings to the index

print("Number of documents in index:", index.ntotal)


Number of documents in index: 4


In [8]:
# Example user query
query = "How to monitor human vital signs in real time?"
query_embedding = model.encode([query])

# Search for the top 3 most similar documents
k = 3
distances, indices = index.search(query_embedding, k)

print("Top matches:")
for i, idx in enumerate(indices[0]):
    print(f"{i + 1}: {corpus[idx]} (Distance: {distances[0][i]})")


Top matches:
1: (IJACSA) International Journal of Advanced Computer Science and Applications, Vol. 7, No. 10, 2016 210 | P a g e www.ijacsa.thesai.org Real Time Monitoring of Human Body Vital Signs using Bluetooth and WLAN Najeed Ahmed Khan Computer Science & IT Department NED University of Engineering & Technology Karachi, Pakistan M. Ajmal Sawand Department of Computer Science Institute of Business Administration Sukkur , Pakistan Marium Hai Electronic Engineering Department NED University of Engineering & Technology Karachi, Pakistan Arwa Khuzema, Mehak Tariq Electronic Engineering Department NED University of Engineering & Technology Karachi, Pakistan Abstract —The technology of telemedicine is emerging and advancing day by day, it is capable of taking the field of healthcare to a whole new level of personalization. A person can keep a c lose check on his/her health's critical signs and can receive suitable feedback if required, - to help maintain the best of health status with the

In [9]:
import gpt4all

In [10]:
dir(gpt4all)

['CancellationError',
 'Embed4All',
 'GPT4All',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_pyllmodel',
 'gpt4all']

In [11]:
from gpt4all import GPT4All

In [13]:
from gpt4all import GPT4All
model = GPT4All("Meta-Llama-3-8B-Instruct.Q4_0.gguf") # downloads / loads a 4.66GB LLM
with model.chat_session():
    print(model.generate("How can I run LLMs efficiently on my laptop?", max_tokens=1024))

Downloading: 100%|██████████| 4.66G/4.66G [09:43<00:00, 7.99MiB/s]
Verifying: 100%|██████████| 4.66G/4.66G [00:21<00:00, 212MiB/s]


Large Language Models (LLMs) are powerful AI models that require significant computational resources to train and use. Running them efficiently on your laptop requires a combination of hardware, software, and optimization techniques. Here's a comprehensive guide to help you get started:

**Hardware Requirements:**

1. **CPU:** A multi-core CPU is essential for parallel processing. Look for at least 4-6 cores (e.g., Intel Core i7 or AMD Ryzen 5).
2. **GPU:** A dedicated graphics card can significantly accelerate computations. Consider a mid-range to high-end GPU like NVIDIA GeForce GTX 1660 Ti, RTX 2060, or AMD Radeon RX 5600 XT.
3. **RAM:** Ensure you have at least 16 GB of RAM (32 GB or more is recommended).
4. **Storage:** A fast SSD (solid-state drive) can improve loading times and overall performance.

**Software Requirements:**

1. **Python:** LLMs are typically implemented in Python, so ensure you have a compatible version installed.
2. **LLM Framework:** Choose an LLM framework 

In [14]:
def generate_answer(query, context):
    """
    Generate an AI answer based on the query and document context using GPT4All.
    """
    # Create a chat session
    with model.chat_session() as session:
        # Generate the response
        prompt = f"Query: {query}\nContext: {context}"
        response = session.generate(prompt, max_tokens=1024)
        return response.strip()

# Example query and context
query = "How does a real-time human vital signs monitoring system work?"
context = "A real-time human vital signs monitoring system captures and analyzes physiological data."

# Generate the answer
answer = generate_answer(query, context)
print("Generated Answer:", answer)

Generated Answer: A real-time human vital signs monitoring system is designed to continuously capture, analyze, and display physiological data from an individual's body. Here's how it typically works:

**Data Acquisition**

1. **Sensors**: The system uses various sensors to collect physiological data, such as:
	* Electrocardiogram (ECG) electrodes for heart rate and rhythm monitoring.
	* Pulse oximetry sensors for oxygen saturation and pulse rate measurement.
	* Blood pressure cuffs or arterial lines for blood pressure monitoring.
	* Temperature probes for body temperature measurement.
2. **Data transmission**: The sensor data is transmitted to a central processing unit (CPU), such as a computer, tablet, or smartphone.

**Real-time Processing**

1. **Signal conditioning**: The CPU processes the raw sensor data by amplifying, filtering, and calibrating it to ensure accurate readings.
2. **Algorithmic analysis**: Advanced algorithms are applied to analyze the processed data in real-time,